# 1. Import Libaries

In [1]:
pip install spotipy sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import spotipy
import requests
from time import sleep

# 2. input client id and client secret

In [3]:
# Prompt the user to enter their Spotify API credentials
#client_id = input('Enter your Spotify client ID: ')
#client_secret = input('Enter your Spotify client secret: ')

In [4]:
client_id = '5fc66d90407b40dca87270453aeeddce'
client_secret = '8bbc6f5427dd4758ad00398414e8ac89'

# 3. pull data

In [5]:
# Get an access token from the Spotify API
response = requests.post('https://accounts.spotify.com/api/token', data={'grant_type': 'client_credentials'}, auth=(client_id, client_secret))
access_token = response.json()['access_token']

In [6]:
# Set up the headers for the HTTP GET request
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

In [7]:
urls = ['https://open.spotify.com/playlist/37i9dQZF1DWUZv12GM5cFk',
      'https://open.spotify.com/playlist/37i9dQZF1DX9Ol4tZWPH6V',
      'https://open.spotify.com/playlist/37i9dQZF1DX0P7PzzKwEKl',
      'https://open.spotify.com/playlist/37i9dQZF1DXaW8fzPh9b08',
      'https://open.spotify.com/playlist/37i9dQZF1DWTWdbR13PQYH',
      'https://open.spotify.com/playlist/37i9dQZF1DWWzQTBs5BHX9',
      'https://open.spotify.com/playlist/37i9dQZF1DX1vSJnMeoy3V',
      'https://open.spotify.com/playlist/37i9dQZF1DX3j9EYdzv2N9',
      'https://open.spotify.com/playlist/37i9dQZF1DWYuGZUE4XQXm',
      'https://open.spotify.com/playlist/37i9dQZF1DX4UkKv8ED8jp',
      'https://open.spotify.com/playlist/37i9dQZF1DXc6IFF23C9jj',
      'https://open.spotify.com/playlist/37i9dQZF1DXcagnSNtrGuJ',
      'https://open.spotify.com/playlist/37i9dQZF1DX0yEZaMOXna3',
      'https://open.spotify.com/playlist/37i9dQZF1DX3Sp0P28SIer',
      'https://open.spotify.com/playlist/37i9dQZF1DX0h0QnLkMBl4',
      'https://open.spotify.com/playlist/37i9dQZF1DX9ukdrXQLJGZ',
      'https://open.spotify.com/playlist/37i9dQZF1DX8XZ6AUo9R4R',
      'https://open.spotify.com/playlist/37i9dQZF1DWTE7dVUebpUW',
      'https://open.spotify.com/playlist/37i9dQZF1DXe2bobNYDtW8',
      'https://open.spotify.com/playlist/37i9dQZF1DWVRSukIED0e9',
      'https://open.spotify.com/playlist/2fmTTbBkXi8pewbUvG3CeZ',
      'https://open.spotify.com/playlist/5GhQiRkGuqzpWZSE7OU4Se',
      'https://open.spotify.com/playlist/56r5qRUv3jSxADdmBkhcz7']

year = []
for i in range(2000,2023):
    year.append(i)

playlists_api = []
for url in urls:
    id = url.split("/")[-1].split("?")[0]
    api = 'https://api.spotify.com/v1/playlists/' + id + '/tracks'
    playlists_api.append(api)
playlists_api = pd.Series(playlists_api, index = year)

In [8]:
TopHits = pd.DataFrame(columns=['playlist_url',
                                'year',
                                'track_id',
                                'track_name', 
                                'track_popularity', 
                                'album',
                                'artist_id',
                                'artist_name', 
                                'artist_genres', 
                                'artist_popularity',  
                                'danceability', 
                                'energy', 
                                'key', 
                                'loudness', 
                                'mode', 
                                'speechiness', 
                                'acousticness', 
                                'instrumentalness', 
                                'liveness', 
                                'valence', 
                                'tempo', 
                                'duration_ms', 
                                'time_signature'])

In [9]:
for playlist_api in playlists_api:
    response = requests.get(playlist_api, headers=headers)
    top_tracks = response.json()['items']
    playlist_url = 'https://open.spotify.com/playlist/' + playlist_api.split("/")[-2].split("?")[0]
    year = playlists_api.index[playlists_api == playlist_api][0]
    for top_track in top_tracks:        
        track_id = top_track['track']['uri'].replace('spotify:track:', '')
        artist_id = top_track['track']['artists'][0]['uri'].replace('spotify:artist:', '')
        artist_info = requests.get('https://api.spotify.com/v1/' + 'artists' + '/' + artist_id, headers=headers)
        artist_info = artist_info.json()
        audio_features = requests.get('https://api.spotify.com/v1/' + 'audio-features' + '/' + track_id, headers=headers)
        
        if audio_features.status_code == 404:
            new_row = pd.Series({'track_id': track_id,
                   'track_name': top_track['track']['name'], 
                   'track_popularity': top_track['track']['popularity'], 
                   'album': top_track['track']['album']['name'],
                   'artist_id': artist_id,
                   'artist_name': top_track['track']['artists'][0]['name'], 
                   'artist_genres': artist_info['genres'], 
                   'artist_popularity': artist_info['popularity']})
        else:
            audio_features = audio_features.json()
            new_row = pd.Series({'track_id': track_id,
                   'track_name': top_track['track']['name'], 
                   'track_popularity': top_track['track']['popularity'], 
                   'album': top_track['track']['album']['name'],
                   'artist_id': artist_id,
                   'artist_name': top_track['track']['artists'][0]['name'], 
                   'artist_genres': artist_info['genres'], 
                   'artist_popularity': artist_info['popularity'],  
                   'danceability': audio_features['danceability'], 
                   'energy': audio_features['energy'], 
                   'key': audio_features['key'], 
                   'loudness': audio_features['loudness'], 
                   'mode': audio_features['mode'], 
                   'speechiness': audio_features['speechiness'], 
                   'acousticness': audio_features['acousticness'], 
                   'instrumentalness': audio_features['instrumentalness'], 
                   'liveness': audio_features['liveness'], 
                   'valence': audio_features['valence'], 
                   'tempo': audio_features['tempo'], 
                   'duration_ms': audio_features['duration_ms'], 
                   'time_signature': audio_features['time_signature']})
        TopHits = pd.concat([TopHits, new_row.to_frame().T], ignore_index=True)
        sleep(0.2)
    values = {'playlist_url': playlist_url, 'year': year}
    TopHits = TopHits.fillna(value=values)

In [10]:
audio_features = requests.get('https://api.spotify.com/v1/' + 'audio-features' + '/' + '37i9dQZEVXbMDoHDwVN2tF', headers=headers)

In [11]:
audio_features

<Response [404]>

In [12]:
TopHits

,playlist_url,year,track_id,track_name,track_popularity,album,artist_id,artist_name,artist_genres,artist_popularity,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000.0,3AJwUDP919kvQ9QcozQPxg,Yellow,91,Parachutes,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,"[permanent wave, pop]",86,...,-7.227,1,0.0281,0.00239,0.000121,0.234,0.285,173.372,266773,4
1,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000.0,2m1hi0nfMR9vdGC8UcrnwU,All The Small Things,84,Enema Of The State,6FBDaR13swtiWwGhX1WQsP,blink-182,"[alternative metal, modern rock, pop punk, pun...",75,...,-4.918,1,0.0488,0.0103,0,0.612,0.684,148.726,167067,4
2,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000.0,3y4LxiYMgDl4RethdzpmNe,Breathe,69,Breathe,25NQNriVT2YbSW80ILRWJa,Faith Hill,"[contemporary country, country, country dawn, ...",61,...,-9.007,1,0.029,0.173,0,0.251,0.278,136.859,250547,4
3,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000.0,60a0Rd6pjrkxjPbaKzXjfq,In the End,88,Hybrid Theory (Bonus Edition),6XyY86QOPPrYVGvF9ch6wz,Linkin Park,"[alternative metal, nu metal, post-grunge, rap...",83,...,-5.87,0,0.0584,0.00958,0,0.209,0.4,105.143,216880,4
4,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000.0,62bOmKYxYg7dhrC6gH9vFn,Bye Bye Bye,74,No Strings Attached,6Ff53KvcvAj5U7Z1vojB5o,*NSYNC,"[boy band, dance pop, pop]",65,...,-4.843,0,0.0479,0.031,0.0012,0.0821,0.861,172.638,200400,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,https://open.spotify.com/playlist/56r5qRUv3jSx...,2022.0,2g6tReTlM2Akp41g0HaeXN,Die Hard,79,Mr. Morale & The Big Steppers,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar,"[conscious hip hop, hip hop, rap, west coast rap]",86,...,-8.072,0,0.247,0.319,0.00116,0.127,0.362,100.988,239027,4
2296,https://open.spotify.com/playlist/56r5qRUv3jSx...,2022.0,4pi1G1x8tl9VfdD9bL3maT,Big Energy,69,777,3MdXrJWsbVzdn6fe5JYkSQ,Latto,[trap queen],85,...,-4.431,0,0.115,0.0453,0,0.341,0.794,106.022,172540,4
2297,https://open.spotify.com/playlist/56r5qRUv3jSx...,2022.0,3LtpKP5abr2qqjunvjlX5i,Doja,86,Doja,5H4yInM5zmHqpKIoMNAx4r,Central Cee,[melodic drill],82,...,-7.43,1,0.288,0.38,0,0.403,0.972,140.04,97393,4
2298,https://open.spotify.com/playlist/56r5qRUv3jSx...,2022.0,3XOalgusokruzA5ZBA2Qcb,pushin P (feat. Young Thug),77,DS4EVER,2hlmm7s2ICUX0LVIhVFlZQ,Gunna,"[atl hip hop, melodic rap, rap, trap]",84,...,-4.572,0,0.187,0.00783,0.00693,0.129,0.488,77.502,136267,1


In [13]:
TopHits.to_csv('playlist_2010to2022',index=False)